<a href="https://colab.research.google.com/github/fowardelcac/Tp2_sem/blob/main/proccess_df_to_ddb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd
import numpy as np

# Preprocesamiento de datos con Python

In [2]:
def process(df: pd.DataFrame):
    # Función para imputar valores faltantes de forma aleatoria
    def random_imputer(df_filter: pd.DataFrame, col: str):
        # Seleccionar valores no nulos de la columna para la imputación
        non_null_values = df_filter[col].dropna().values

        # Aplicar un valor aleatorio de non_null_values donde hay NaN en la columna
        df_filter.loc[:, col] = df_filter[col].apply(
            lambda x: np.random.choice(non_null_values) if pd.isnull(x) else x
        )
        return df_filter

    # Crear una copia del DataFrame original
    df = df.copy()
    df.drop_duplicates(inplace=True)  # Eliminar duplicados
    df_filter = df.dropna(
        subset=["country", "designation"]
    )  # Filtrar filas con valores nulos en las columnas clave
    df_filter = random_imputer(
        df_filter, "price"
    )  # Imputar valores faltantes en la columna "price"
    return df_filter

In [30]:
def build_country(df: pd.DataFrame):
  """
    Crea un DataFrame con una lista única de países, asignando un ID a cada país.

    Parámetros:
        df (pd.DataFrame): DataFrame original que contiene una columna 'country'.

    Devuelve:
        countries_df (pd.DataFrame): DataFrame con columnas 'country' y 'country_id'.
    """
  unique_countries = df['country'].dropna().unique()
  countries_df = pd.DataFrame(unique_countries, columns=['country'])
  countries_df['country_id'] = countries_df.index + 1
  return countries_df

def build_prov(df: pd.DataFrame, country_df: pd.DataFrame):
  """
    Crea un DataFrame con una lista única de provincias y sus correspondientes países.

    Parámetros:
        df (pd.DataFrame): DataFrame original que contiene las columnas 'province' y 'country'.
        country_df (pd.DataFrame): DataFrame de países, generado por build_country.

    Devuelve:
        provinces_df (pd.DataFrame): DataFrame con columnas 'province', 'country_id' y 'province_id'.
    """
  unique_provinces = df[['province', 'country']].dropna().drop_duplicates()  # Obtener provincias con su país
  unique_provinces = unique_provinces.merge(country_df[['country_id', 'country']], on='country', how='left')  # Unir con la tabla de países
  provinces_df = unique_provinces[['province', 'country_id']]  # Seleccionar las columnas necesarias
  provinces_df['province_id'] = provinces_df.index + 1
  provinces_df.columns = ['province', 'country_id', 'province_id',]
  return provinces_df # Renombrar las columnas para mantener consistencia

def build_region(df: pd.DataFrame, province_df: pd.DataFrame):
  """
    Crea un DataFrame con una lista única de regiones y sus correspondientes provincias.

    Parámetros:
        df (pd.DataFrame): DataFrame original que contiene las columnas 'province' y 'region_1'.
        province_df (pd.DataFrame): DataFrame de provincias, generado por build_prov.

    Devuelve:
        region_df (pd.DataFrame): DataFrame con columnas 'region', 'province_id' y 'region_id'.
    """
  unique_region = df[['province', 'region_1']].dropna().drop_duplicates()
  unique_region = unique_region.merge(province_df[['province_id', 'province']], on='province', how='left')
  region_df = unique_region[['region_1', 'province_id']]
  region_df['region_id'] = region_df.index + 1
  region_df.columns = ['region', 'province_id', 'region_id']
  return region_df

def build_variety(df: pd.DataFrame):
  """
    Crea un DataFrame con una lista única de variedades de vino y asigna un ID único a cada una.

    Parámetros:
        df (pd.DataFrame): DataFrame original que contiene la columna 'variety'.

    Devuelve:
        var_df (pd.DataFrame): DataFrame con columnas 'variety' y 'variety_id'.
    """
  unique_varieties = df['variety'].dropna().unique()
  var_df = pd.DataFrame(unique_varieties, columns=['variety'])
  var_df['variety_id'] = var_df.index + 1
  return var_df

def build_winery(df: pd.DataFrame):
  """
    Crea un DataFrame con una lista única de bodegas y asigna un ID único a cada una.

    Parámetros:
        df (pd.DataFrame): DataFrame original que contiene la columna 'winery'.

    Devuelve:
        win_var (pd.DataFrame): DataFrame con columnas 'winery' y 'winery_id'.
    """
  unique_winery = df['winery'].dropna().unique()
  win_var = pd.DataFrame(unique_winery, columns=['winery'])
  win_var['winery_id'] = win_var.index + 1
  return win_var

def build_taster(df: pd.DataFrame):
  """
    Crea un DataFrame con una lista única de catadores y sus nombres en Twitter.

    Parámetros:
        df (pd.DataFrame): DataFrame original que contiene las columnas 'taster_name' y 'taster_twitter_handle'.

    Devuelve:
        taster_df (pd.DataFrame): DataFrame con columnas 'taster_name', 'taster_twitter_handle' y 'taster_id'.
    """
  unique_taster = df['taster_name'].dropna().unique()
  taster_df = pd.DataFrame(unique_taster, columns=['taster_name'])

  taster_handles = df[['taster_name', 'taster_twitter_handle']].dropna().drop_duplicates()
  taster_df = taster_df.merge(taster_handles, on='taster_name', how='left')
  taster_df['taster_id'] = taster_df.index + 1
  return taster_df

def build_wine(df: pd.DataFrame, country_table: pd.DataFrame, province_table: pd.DataFrame, region_table: pd.DataFrame, variety_table: pd.DataFrame, winery_table: pd.DataFrame):
  """
    Crea un DataFrame de vinos, cada uno con información relevante y un ID único.

    Parámetros:
        df (pd.DataFrame): DataFrame original que contiene columnas relacionadas con vinos.
        country_table (pd.DataFrame): DataFrame de países, generado por build_country.
        province_table (pd.DataFrame): DataFrame de provincias, generado por build_prov.
        region_table (pd.DataFrame): DataFrame de regiones, generado por build_region.
        variety_table (pd.DataFrame): DataFrame de variedades, generado por build_variety.
        winery_table (pd.DataFrame): DataFrame de bodegas, generado por build_winery.

    Devuelve:
        wine_df (pd.DataFrame): DataFrame de vinos con columnas seleccionadas y 'wine_id'.
    """
  wine_df = df[['title', 'designation', 'points', 'price', 'variety', 'winery', 'country']]
  wine_df = wine_df.merge(variety_table[["variety_id", "variety"]], on='variety', how='left')
  wine_df = wine_df.merge(winery_table[["winery_id", "winery"]], on='winery', how='left')
  wine_df = wine_df.merge(country_table[["country_id", "country"]], on='country', how='left')
  wine_df.drop(['variety', 'winery', 'country'], axis=1, inplace=True)
  wine_df['wine_id'] = wine_df.index + 1
  return wine_df

def build_review(df: pd.DataFrame, taster_table: pd.DataFrame, wine_table: pd.DataFrame):
  """
    Crea un DataFrame con las reseñas de los vinos, asociando cada reseña a un catador y un vino.

    Parámetros:
        df (pd.DataFrame): DataFrame original con las columnas 'taster_name', 'title' y 'description'.
        taster_table (pd.DataFrame): DataFrame de catadores, generado por build_taster.
        wine_table (pd.DataFrame): DataFrame de vinos, generado por build_wine.

    Devuelve:
        review_table (pd.DataFrame): DataFrame de reseñas con columnas 'taster_id', 'wine_id' y 'review_id'.
    """
  review_table = df[['taster_name', 'title', 'description']]
  review_table = review_table.merge(taster_table[["taster_id", "taster_name"]], on='taster_name', how='left')
  review_table['taster_id'] = review_table['taster_id'].fillna(0).astype(int)

  review_table = review_table.merge(wine_table[["wine_id", "title"]], on='title', how='left')
  review_table.drop(["title", "taster_name"], axis=1, inplace=True)
  review_table['review_id'] = review_table.index + 1
  return review_table

In [4]:
def processor():
  """
    Procesa los datos del archivo CSV de vinos y genera tablas separadas para cada entidad (país, provincia, región, etc.)
    con claves únicas. La función crea tablas a partir de datos originales de vinos, catadores, bodegas y más,
    asignando índices a cada registro único.

    Devuelve:
        tuple: Un conjunto de DataFrames para las siguientes tablas:
            - country_table: Tabla de países con 'country_id' como índice.
            - province_table: Tabla de provincias con 'province_id' como índice.
            - region_table: Tabla de regiones con 'region_id' como índice.
            - variety_table: Tabla de variedades de vino con 'variety_id' como índice.
            - winery_table: Tabla de bodegas con 'winery_id' como índice.
            - taster_table: Tabla de catadores con 'taster_id' como índice.
            - wine_table: Tabla de vinos con 'wine_id' como índice.
            - review_table: Tabla de reseñas con 'review_id' como índice.
    """
  data = pd.read_csv("https://media.githubusercontent.com/media/fowardelcac/Tp2_sem/refs/heads/main/winemag-data-130k-v2.csv").drop('Unnamed: 0', axis = 1)
  df = process(data)

  country_table = build_country(df)
  province_table = build_prov(df, country_table)
  region_table = build_region(df, province_table)
  variety_table = build_variety(df)
  winery_table = build_winery(df)
  taster_table = build_taster(df)
  taster_table.loc[len(taster_table)] = ['unknown', 'NaN', 0]

  wine_table = build_wine(df, country_table, province_table, region_table, variety_table, winery_table)
  review_table = build_review(df, taster_table, wine_table)
  return country_table.set_index("country_id"), province_table.set_index("province_id"), region_table.set_index("region_id"), variety_table.set_index("variety_id"), winery_table.set_index("winery_id"), taster_table.set_index("taster_id"), wine_table.set_index("wine_id"), review_table.set_index("review_id")


In [5]:
country_table, province_table, region_table, variety_table, winery_table, taster_table, wine_table, review_table = processor()

In [6]:
tables_list = [country_table, province_table, region_table, variety_table, winery_table, taster_table, wine_table, review_table]
for i in tables_list:
  print(i.head(1))
  print("*"* 100)

           country
country_id        
1            Italy
****************************************************************************************************
                      province  country_id
province_id                               
1            Sicily & Sardinia           1
****************************************************************************************************
          region  province_id
region_id                    
1           Etna            1
****************************************************************************************************
                variety
variety_id             
1           White Blend
****************************************************************************************************
            winery
winery_id         
1          Nicosia
****************************************************************************************************
             taster_name taster_twitter_handle
taster_id                                  

# SQL

In [32]:
# Lista de nombres de tablas que se utilizarán en la base de datos
table_names = ['country', 'province', 'region', 'variety', 'winery', 'taster', 'wine', 'review']

# Conectar a (o crear) una base de datos SQLite llamada 'WINE_DDB.db'
conexion = sqlite3.connect('WINE_DDB.db')

# Insertar cada DataFrame en la base de datos como una tabla
with conexion:
    for i, table_name in enumerate(table_names):
        # Guardar cada DataFrame en la base de datos SQLite
        tables_list[i].to_sql(name=table_name, con=conexion, if_exists='replace', index=True)

Primero realizo un SELECT a cada tabla para visualizar si estan correctamente construidas

In [33]:
# Ejecutar una consulta para obtener los primeros 3 registros de la tabla 'country' y almacenarlos en un DataFrame
pd.read_sql(f"SELECT * FROM country LIMIT 3;", conexion)

,country_id,country
0,1,Italy
1,2,Portugal
2,3,US


In [9]:
pd.read_sql(f"SELECT * FROM {table_names[1]} LIMIT 3;", conexion)

,province_id,province,country_id
0,1,Sicily & Sardinia,1
1,2,Douro,2
2,3,Michigan,3


In [10]:
pd.read_sql(f"SELECT * FROM {table_names[2]} LIMIT 3;", conexion)

,region_id,region,province_id
0,1,Etna,1
1,2,Lake Michigan Shore,3
2,3,Willamette Valley,4


In [11]:
pd.read_sql(f"SELECT * FROM {table_names[3]} LIMIT 3;", conexion)

,variety_id,variety
0,1,White Blend
1,2,Portuguese Red
2,3,Riesling


In [12]:
pd.read_sql(f"SELECT * FROM {table_names[4]} LIMIT 3;", conexion)

,winery_id,winery
0,1,Nicosia
1,2,Quinta dos Avidagos
2,3,St. Julian


In [13]:
pd.read_sql(f"SELECT * FROM {table_names[5]} LIMIT 3;", conexion)

,taster_id,taster_name,taster_twitter_handle
0,1,Kerin O’Keefe,@kerinokeefe
1,2,Roger Voss,@vossroger
2,3,Alexander Peartree,None


In [14]:
pd.read_sql(f"SELECT * FROM {table_names[6]} LIMIT 3;", conexion)

,wine_id,title,designation,points,price,variety_id,winery_id,country_id
0,1,Nicosia 2013 Vulkà Bianco (Etna),Vulkà Bianco,87,35.0,1,1,1
1,2,Quinta dos Avidagos 2011 Avidagos Red (Douro),Avidagos,87,15.0,2,2,2
2,3,St. Julian 2013 Reserve Late Harvest Riesling ...,Reserve Late Harvest,87,13.0,3,3,3


In [15]:
pd.read_sql(f"SELECT * FROM {table_names[7]} LIMIT 3;", conexion)

,review_id,description,taster_id,wine_id
0,1,"Aromas include tropical fruit, broom, brimston...",1,1
1,2,"This is ripe and fruity, a wine that is smooth...",2,2
2,3,"Pineapple rind, lemon pith and orange blossom ...",3,3


# Consultas SQL

# Obtener la cantidad de vinos por pais.
Este código ejecuta una consulta SQL para obtener los diez países con el mayor número de vinos en la base de datos. La consulta agrupa los vinos por país, cuenta cuántos vinos hay en cada país y luego ordena los resultados de mayor a menor, limitando la salida a los diez primeros.

In [16]:
query1 = """
SELECT country, COUNT(wine_id) AS total_wines
FROM wine
JOIN country  ON wine.country_id = country.country_id
GROUP BY country
ORDER BY total_wines DESC LIMIT 10;
"""
pd.read_sql(query1, conexion)

,country,total_wines
0,US,34170
1,France,13386
2,Italy,12798
3,Spain,4891
4,Portugal,4715
5,Chile,3627
6,Argentina,2693
7,Austria,2582
8,Germany,1847
9,Australia,1539


# Precio promedio por vino
Este código ejecuta una consulta SQL para obtener el precio promedio de los vinos agrupados por puntuaciones, ordenando los resultados de mayor a menor según las puntuaciones.

In [17]:
query2 = """
SELECT points, AVG(price) AS avg_price
FROM wine
GROUP BY points
ORDER BY points DESC;
"""
pd.read_sql(query2, conexion)

,points,avg_price
0,100,359.692308
1,99,257.482759
2,98,202.622642
3,97,146.781065
4,96,124.163070
5,95,94.205191
6,94,75.791284
7,93,61.279630
8,92,51.155464
9,91,44.331797


# Los 10 sommelier con mas reseñas

In [18]:
query = """
SELECT taster_name, COUNT(*) AS total_reviews
FROM taster
ORDER BY total_reviews DESC
"""
pd.read_sql(query, conexion)

,taster_name,total_reviews
0,Kerin O’Keefe,20


In [19]:
query = """
SELECT *
FROM taster
"""
pd.read_sql(query, conexion)

,taster_id,taster_name,taster_twitter_handle
0,1,Kerin O’Keefe,@kerinokeefe
1,2,Roger Voss,@vossroger
2,3,Alexander Peartree,None
3,4,Paul Gregutt,@paulgwine
4,5,Michael Schachner,@wineschach
5,6,Anna Lee C. Iijima,None
6,7,Virginie Boone,@vboone
7,8,Matt Kettmann,@mattkettmann
8,9,Sean P. Sullivan,@wawinereport
9,10,Joe Czerwinski,@JoeCz


In [20]:
query = """
SELECT taster_id, COUNT(*) AS total_reviews
FROM review
WHERE taster_id != 0
GROUP BY taster_id
ORDER BY total_reviews DESC
LIMIT 10;
"""
pd.read_sql(query, conexion)


,taster_id,total_reviews
0,2,17738
1,5,11693
2,1,6904
3,7,6304
4,4,6080
5,8,4195
6,10,3219
7,9,3215
8,6,3196
9,16,2872


Este código ejecuta una consulta SQL para obtener los diez catadores que han realizado el mayor número de reseñas en la base de datos.

In [21]:
query = """
SELECT
    t.taster_name,
    COUNT(*) AS total_reviews
FROM review r
JOIN taster t ON r.taster_id = t.taster_id
WHERE r.taster_id != 0
GROUP BY t.taster_name
ORDER BY total_reviews DESC
LIMIT 10;

"""
pd.read_sql(query, conexion)


,taster_name,total_reviews
0,Roger Voss,17738
1,Michael Schachner,11693
2,Kerin O’Keefe,6904
3,Virginie Boone,6304
4,Paul Gregutt,6080
5,Matt Kettmann,4195
6,Joe Czerwinski,3219
7,Sean P. Sullivan,3215
8,Anna Lee C. Iijima,3196
9,Anne Krebiehl MW,2872


#Variedad de uva mas comun
Este código ejecuta una consulta SQL que obtiene las cinco variedades de vino más comunes en la base de datos, junto con el conteo de cuántos vinos hay de cada variedad.

In [22]:
query = """
SELECT
    wine.variety_id,
    variety.variety,
    COUNT(*) AS variety_count
FROM wine
JOIN variety ON wine.variety_id = variety.variety_id
GROUP BY wine.variety_id, variety.variety
ORDER BY variety_count DESC
LIMIT 5;
"""
pd.read_sql(query, conexion)

,variety_id,variety,variety_count
0,4,Pinot Noir,9155
1,23,Chardonnay,7263
2,13,Red Blend,6874
3,9,Cabernet Sauvignon,5415
4,3,Riesling,4041


# Bodegas con mas produccion
Este código ejecuta una consulta SQL que obtiene las cinco bodegas más representadas en la base de datos, junto con el conteo de cuántos vinos pertenecen a cada bodega

In [23]:
query = """
SELECT
    wine.winery_id,
    winery.winery,
    COUNT(*) AS winery_count
FROM wine
JOIN winery ON wine.winery_id = winery.winery_id
GROUP BY wine.winery_id, winery.winery_id
ORDER BY winery_count DESC
LIMIT 5;
"""
pd.read_sql(query, conexion)

,winery_id,winery,winery_count
0,493,Wines & Winemakers,211
1,949,DFJ Vinhos,200
2,97,Testarossa,188
3,704,Williams Selyem,178
4,551,Georges Duboeuf,171


# Paises con vinos mas caros
Este código ejecuta una consulta SQL que obtiene las cinco países con el precio máximo de vino, junto con el identificador y el nombre del país.

In [24]:
query = """
SELECT
    wine.country_id,
    country.country_id,
    country.country,
    MAx(wine.price) AS max_price
FROM wine
JOIN country ON wine.country_id = country.country_id
GROUP BY wine.country_id, country.country_id
ORDER BY max_price DESC
LIMIT 5;
"""
pd.read_sql(query, conexion)

,country_id,country_id,country,max_price
0,3,3,US,2013.0
1,13,13,Austria,1100.0
2,2,2,Portugal,1000.0
3,1,1,Italy,900.0
4,9,9,Australia,850.0


# Paises con vinos de mejor puntuacion en promedio
Este código ejecuta una consulta SQL que obtiene los cinco países con la puntuación promedio más alta de sus vinos.

In [26]:
query = """
SELECT
    wine.country_id,
    country.country_id,
    country.country,
    AVG(wine.points) AS avg_points
FROM wine
JOIN country ON wine.country_id = country.country_id
GROUP BY wine.country_id, country.country_id
ORDER BY avg_points DESC
LIMIT 5;
"""
pd.read_sql(query, conexion)

,country_id,country_id,country,avg_points
0,23,23,England,91.746032
1,29,29,India,91.500000
2,13,13,Austria,90.398916
3,5,5,Germany,90.065512
4,12,12,Hungary,89.477477
